Version: 2.00.006

This Python script retrieves solution variables from cplex.

TO-DO:
* Esta version depende del fichero en formato LP que ha sido generado por Gusek. Desde `Gusek` se ha generado previamente el modelo `.lp`. Para ello se va al menú de Gusek y se ejecuta: `Tools > Build Cplex LP`.
    * Sería interesante crear el modelo y los datos desde Python. Esto evitaría depender de Gusek y tendríamos un modelo con un formato adecuado.


### IMPORTANTE:

* Hay que tener `cplex` instalado.
* Para poder usar `cplex`, antes hay que abrir un "Anaconda prompt" desde el menu de Windows y ejecutar alli la instruccion: `conda install -c ibmdecisionoptimization cplex`


# Resolución del problema con `cplex`.

Desde `Gusek` se ha generado previamente el modelo `.lp`. Para ello se va al menú de Gusek y se ejecuta: `Tools > Build Cplex LP`.

Ahora se resuelve ese modelo `.MPS` usando `cplex` y se deposita el resultado en el objeto `cpx`.



In [1]:
# cplex info
# https://www.ibm.com/support/knowledgecenter/en/SSSA5P_12.4.0/ilog.odms.cplex.help/refpythoncplex/html/cplex.Cplex-class.html

import cplex

cpx = cplex.Cplex()
cpx.read('md.mc.ml.lp')
cpx.solve()


CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 121 rows and 63 columns.
MIP Presolve modified 351 coefficients.
Reduced MIP has 252 rows, 300 columns, and 1590 nonzeros.
Reduced MIP has 282 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.31 ticks)
Probing time = 0.00 sec. (0.27 ticks)
Tried aggregator 1 time.
Reduced MIP has 252 rows, 300 columns, and 1590 nonzeros.
Reduced MIP has 282 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.99 ticks)
Probing time = 0.00 sec. (0.28 ticks)
Clique table members: 99.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.88 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0      230.4900    25         

# Leer fichero `.dat`
Leer el fichero .dat y volcar los parametros de ese fichero en variables de Python. Luego se usarán esas variables para hacer los cálculos.

In [2]:
import numpy as np

# Leer el fichero .dat y volcar los valores de los parametros en variables.

# Se abre el fichero con los datos
filepath = 'md.mc.ml.DAT'


with open(filepath, 'r') as fp:
    # Se recorre linea a linea
    line = fp.readline()
    while line:
        # Se separan las palabras de cada linea
        elementos = line.replace('\n',' ').split()
        # Se esta line no tiene nada, salta a la siguiente line
        if (len(elementos) == 0):
            line = fp.readline()
            continue 
            
        # Se buscan los parametros:
        # la primera palabra es 'param' y en algun lugar se asigna su valor con ':='
        if (elementos[0] == 'param'):
            nElemento = 1
            # print('\n', line, 'len(elementos)', len(elementos))
            # Se busca dendtro de la linea, en que posicion esta la asignacion ':='
            while (elementos[nElemento] != ':='):
                nElemento += 1
            # print('nElemento = ', nElemento, elementos[nElemento-1])
            # El elemento anterior es el parametro buscado 
            # y el elemento posterior contiene su valor
            
            # Se asigna el valor del parametro a la variable correspondiente.
            if (elementos[nElemento-1] == 'precio_por_km'):
                # se selecciona el valor quitando el simbolo ";"
                precio_por_km = float(elementos[nElemento+1][:-1])
            elif (elementos[nElemento-1] == 'num_depositos'):
                # se selecciona el valor quitando el simbolo ";" del final
                num_depositos  = int(elementos[nElemento+1][:-1])
                # ND el conjunto de depositos
                ND = range(0,num_depositos)
            elif (elementos[nElemento-1] == 'num_clientes'):
                # se selecciona el valor quitando el simbolo ";" del final
                num_clientes   = int(elementos[nElemento+1][:-1])
                # Nv_x el conjunto de clientes. O sea los indices continuan
                # desde (num_depositos) hasta (num_depositos + num_clientes)
                Nv_x = range(num_depositos, num_depositos + num_clientes)
                # Nv_y_d Se usa para definir los indices de la variable y_ijk.
                # Se usan estos mismos indices para la variable d_jg
                Nv_y_d = range(0, num_clientes)
                # Ntot el conjunto de todos los nodos denotado por los indices i y j
                # O sea ND y Nv son subconjuntos disjuntos de Ntot
                Ntot = range(0, num_depositos + num_clientes)
            elif (elementos[nElemento-1] == 'num_productos'):
                # se selecciona el valor quitando el simbolo ";" del final
                num_productos = int(elementos[nElemento+1][:-1])
                # G el conjunto de productos; denotado por el indice g
                G = range(0, num_productos)
            elif (elementos[nElemento-1] == 'num_vehiculos'):
                # se selecciona el valor quitando el simbolo ";" del final
                num_vehiculos = int(elementos[nElemento+1][:-1])
                print('num_vehiculos = ', num_vehiculos)
                # K el conjunto de vehiculos; denotado por el indice k
                K = range(0, num_vehiculos)
            elif (elementos[nElemento-1] == 'MC'):
                # MC la distancia maxima, que cada vehiculo se permite viajar
                # se selecciona el valor quitando el simbolo ";" del final
                MC = int(elementos[nElemento+1][:-1])
            elif (elementos[nElemento-1] == 'c'):
                # c_ij la distancia entre los nodos i y j
                # c es una matriz con los datos de las distancias entre todos los nodos
                # Se define el numero total de nodos
                num_nodos = num_depositos + num_clientes
                # Se define la matriz c de las distancias y se rellena de ceros
                c = np.array(np.zeros((num_nodos, num_nodos), dtype=int))

                # Se recorre el fichero para ir rellenando valores de esta matriz
                # Se lee la siguiente linea y
                # Se separan las palabras de cada linea
                elementos = fp.readline().replace('\n',' ').split()
                while (elementos[len(elementos)-1] != ';'):
                    # Se asigna el dato a la matriz
                    c[int(elementos[0]) -1, int(elementos[1]) -1] = int(elementos[2])
                    # Se salta a la siguiente linea
                    elementos = fp.readline().replace('\n',' ').split()
                print('c[i,j]:\n', c)
            elif (elementos[nElemento-1] == 'd'):
                # d_jg demanda del nodo j para el producto g
                # d es una matriz con los datos de las demandas de los clientes para cada producto
                # Se define la matriz d y se rellena de ceros
                # Ahora el primer cliente tiene el indice 0
                d = np.array(np.zeros((num_clientes, num_productos), dtype=int))
                
                # Se recorre el fichero para ir rellenando valores de esta matriz
                # Se lee la siguiente linea y
                # Se separan las palabras de cada linea
                elementos = fp.readline().replace('\n',' ').split()
                while (elementos[len(elementos)-1] != ';'):
                    # Se asigna el dato a la matriz
                    d[int(elementos[0]) - num_depositos - 1, int(elementos[1]) - 1] = int(elementos[2])
                    # Se salta a la siguiente linea
                    elementos = fp.readline().replace('\n',' ').split()
                print('d[j,g]:\n',d)
            elif (elementos[nElemento-1] == 'Q'):
                # Q_g la capacidad del compartimento de cualquier vehiculo dedicado al producto g
                # Se define la matriz Q y se rellena de ceros
                Q = np.array(np.zeros((num_productos, 1), dtype=int))
                
                # Se recorre el fichero para ir rellenando valores de esta matriz
                # Se lee la siguiente linea y
                # Se separan las palabras de cada linea
                elementos = fp.readline().replace('\n',' ').split()
                while (elementos[len(elementos)-1] != ';'):
                    # Se asigna el dato a la matriz
                    Q[int(elementos[0]) - 1] = int(elementos[1])
                    # Se salta a la siguiente linea
                    elementos = fp.readline().replace('\n',' ').split()
                print('Q[g]:\n',Q)
            elif (elementos[nElemento-1] == 'DQ'):
                # DQ_ik la lista de vehiculos k que salen del deposito i
                # DQ[i,k] es una matriz 
                # Se define la matriz DQ de los vehiculos de cada deposito
                DQ = np.array(np.zeros((num_depositos, num_vehiculos), dtype=int))
                
                # Se recorre el fichero para ir rellenando valores de esta matriz
                # Se lee la siguiente linea y
                # Se separan las palabras de cada linea
                elementos = fp.readline().replace('\n',' ').split()
                while (elementos[len(elementos)-1] != ';'):
                    # Se asigna el dato a la matriz
                    DQ[int(elementos[0]) -1, int(elementos[1]) -1] = int(elementos[2])
                    # Se salta a la siguiente linea
                    elementos = fp.readline().replace('\n',' ').split()
                print('DQ[i,k]:\n', DQ)
            elif (elementos[nElemento-1] == 'S'):
                # S_ig = stock en el deposito i del producto g
                # S[i,g] es una matriz 
                # Se define la matriz S de cada deposito con su stock de cada producto
                S = np.array(np.zeros((num_depositos, num_productos), dtype=int))
                
                # Se recorre el fichero para ir rellenando valores de esta matriz
                # Se lee la siguiente linea y
                # Se separan las palabras de cada linea
                elementos = fp.readline().replace('\n',' ').split()
                while (elementos[len(elementos)-1] != ';'):
                    # Se asigna el dato a la matriz
                    S[int(elementos[0]) -1, int(elementos[1]) -1] = int(elementos[2])
                    # Se salta a la siguiente linea
                    elementos = fp.readline().replace('\n',' ').split()
                print('S[i,g]:\n', S)
            elif (elementos[nElemento-1] == 'fk'):
                # fk el costo fijo de usar el vehiculo k

                # Se define la matriz fk y se rellena de ceros
                fk = np.array(np.zeros((num_vehiculos, 1), dtype=int))
                
                # Se recorre el fichero para ir rellenando valores de esta matriz
                # Se lee la siguiente linea y
                # Se separan las palabras de cada linea
                elementos = fp.readline().replace('\n',' ').split()
                while (elementos[len(elementos)-1] != ';'):
                    # Se asigna el dato a la matriz
                    fk[int(elementos[0]) - 1] = int(elementos[1])
                    # Se salta a la siguiente linea
                    elementos = fp.readline().replace('\n',' ').split()
                print('fk[k]:\n',fk)

            
        # Se busca en la siguiente linea
        line = fp.readline()
# parametros
# cpx.parameters()

num_vehiculos =  3
c[i,j]:
 [[  0  30  20 262 180 306 140 117 216]
 [ 30   0  10 237 154 281 115  92 191]
 [ 20  10   0 220 170 264 100  66 206]
 [262 237 220   0 179  50 123 276  42]
 [180 154 170 179   0 222  62 195 133]
 [306 281 264  50 222   0 167 320  86]
 [140 115 100 123  62 167   0 156  87]
 [117  92  66 276 195 320 156   0 231]
 [216 191 206  42 133  86  87 231   0]]
d[j,g]:
 [[10 11  3 18  0]
 [10  5  2  0 13]
 [ 0 42 14 29 44]
 [27  0  0 23 41]
 [ 8 25 19  0 26]
 [30 20 13 28  0]]
Q[g]:
 [[50]
 [75]
 [35]
 [78]
 [85]]
DQ[i,k]:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
S[i,g]:
 [[ 70 100  50 100 100]
 [ 70 100  50 100 100]
 [ 70 100  50 100 100]]
fk[k]:
 [[100]
 [100]
 [100]]


# Se lee la solucion del problema que ha generado `cplex`.
Una vez resuelto el problema se lee la solución y se guardan los datos en las matrices correspondientes.

In [3]:
# Se crea la matriz x[i,j,k] y se llena de ceros.
# x_ijk equivalen a 1 si la ruta entre los nodos i y j es recorrida por el vehiculo k, y es cero en caso contrario
# Segun la restriccion 13, x_ijk es binaria
# var x{i in Ntot, j in Ntot, k in K}, binary;

x = np.array(np.zeros((num_nodos, num_nodos, num_vehiculos), dtype=int))


# Se crea la matriz y[j,g,k] y se llena de ceros.
# y_jgk igual a 1 si la demanda del nodo j para el producto g es entregada por el vehiculo k;es cero de lo contrario
# Segun la restriccion 13, y_igk es binaria
# var y{j in Nv, g in G, k in K}, binary;
y = np.array(np.zeros((num_clientes, num_productos, num_vehiculos), dtype=int))


# Se crea la matriz u[k] y se llena de ceros.
# u_k igual a 1 si el vehiculo k es usado, 0 en caso contario
# var u{k in K}, binary;
u = np.array(np.zeros((num_vehiculos, 1), dtype=int))


# Check solution status here via cpx.solution.get_status()
for name, value in zip(cpx.variables.get_names(),
                       cpx.solution.get_values()):
    # Se seleccionan todas las soluciones distintas de 0 y se asigna en el array correspondiente
    ####if (value != 0.0):
    # Las variables usadas en este caso son binarias
    if (abs(value) > 0.8):
        # Se muestran los datos de Cplex
        print (name, value)
        # Se asignan los datos de Cplex a las variables de Python
        if (name[0] == 'x'):
            # x[i,j,k]
            # Se buscan los indices que estan entre parentesis
            idx_ini = name.index('(')
            idx_fin = name.index(')')
            # Se separan los numeros, que estan en formato string
            # Los indices estan separados por comas ','
            indices_str = name[idx_ini+1:idx_fin].split(',')
            x[int(indices_str[0]) - 1, int(indices_str[1]) - 1, int(indices_str[2]) - 1] = 1
        ## print ('x[i,j,k]:\n', x)
        elif (name[0] == 'y'):
            # y[j,g,k]
            # Se buscan los indices que estan entre parentesis
            idx_ini = name.index('(')
            idx_fin = name.index(')')
            # Se separan los numeros, que estan en formato string
            indices_str = name[idx_ini+1:idx_fin].split(',')
            #### print('INDICES y: ', indices_str[0], indices_str[1], indices_str[2])
            y[int(indices_str[0]) - num_depositos - 1, 
              int(indices_str[1]) - 1, 
              int(indices_str[2]) - 1] = 1
        elif (name[0] == 'u'):
            # u[k]
            # Se busca el indice que esta entre parentesis
            idx_ini = name.index('(')
            idx_fin = name.index(')')
            indices_str = name[idx_ini+1:idx_fin]
            u[int(indices_str[0]) - 1, 0] = 1
print ('u[k]:\n', u)
## for j in Nv:
##     for g in G:
##         for k in K:
##             if (y[j,g,k] != 0):
##                 print('y[',j,',',g,',',k,')', y[j,g,k])
                


x(2,9,2) 1.0
x(3,2,2) 1.0
x(3,7,3) 1.0
x(4,3,2) 1.0
x(5,8,3) 1.0
x(6,4,2) 1.0
x(7,5,3) 1.0
x(8,3,3) 1.0
x(9,6,2) 1.0
u(2) 1.0
u(3) 1.0
y(4,1,2) 1.0
y(4,2,2) 1.0
y(4,3,2) 1.0
y(4,4,2) 1.0
y(4,5,2) 1.0
y(5,1,3) 1.0
y(5,2,3) 1.0
y(5,3,3) 1.0
y(5,4,3) 1.0
y(5,5,3) 1.0
y(6,1,2) 1.0
y(6,2,2) 0.9999999999999991
y(6,3,2) 0.9999999999999991
y(6,4,2) 1.0
y(6,5,2) 1.0
y(7,1,3) 1.0
y(7,2,3) 1.0
y(7,3,3) 1.0
y(7,4,3) 1.0
y(7,5,3) 1.0
y(8,1,3) 1.0
y(8,2,3) 1.0
y(8,3,3) 1.0
y(8,4,3) 1.0
y(8,5,3) 1.0
y(9,1,2) 1.0
y(9,2,2) 1.0
y(9,3,2) 1.0000000000000009
y(9,4,2) 1.0
y(9,5,2) 1.0
ST(4,2) 3.0
ST(5,2) 65.0
ST(5,3) 2.0
ST(6,2) 2.0
ST(7,3) 1.0
ST(8,3) 3.0
ST(9,2) 1.0
u[k]:
 [[0]
 [1]
 [1]]


# Mostrar los resultados.

Con ayuda de las matrices que se han guardado anteriormente, se muestran los resultados y se hacen los calculos de las rutas

## Función objetivo

In [4]:
distancia_total = 0
gasto_vehiculos = 0


# Se calcula el gasto en los kilometros recorridos
for i in Ntot:
    for j in Ntot:
        for k in K:
            distancia_total += c[i,j] * x[i,j,k]

# Se calcula el gasto de los vehiculos
for k in K:
    gasto_vehiculos += fk[k] * u[k]

resultado = precio_por_km * distancia_total + gasto_vehiculos
print('Resultado de la funcion objetivo (gasto) = ', resultado)
print('El camino optimo recorre una distancia = ', distancia_total)
print('El recorrido optimo tiene gasto fijo en los vehiculos = ', gasto_vehiculos)
print('El precio por km es de = ', precio_por_km)


Resultado de la funcion objetivo (gasto) =  [582.2]
El camino optimo recorre una distancia =  980
El recorrido optimo tiene gasto fijo en los vehiculos =  [200]
El precio por km es de =  0.39


## Gastos por vehículo y distancias

In [5]:
print('El recorrido optimo tiene gasto fijo en los vehiculos = ', gasto_vehiculos)

# Se desglosa el gasto fijo de cada vehiculo
for k in K:
    if (u[k] != 0):
        print('Vehiculo', k+1, '. Coste fijo = ', fk[k])

print('\nLa distancia maxima permitida para un vehiculo es = ', MC)

for k in K:
    distancia_aux = 0
    # Se muestran los datos si se usa ese vehiculo
    if (u[k] != 0):
        for i in Ntot:
            for j in Ntot:
                distancia_aux += c[i,j] * x[i,j,k]
    print('La distancia recorrida por el vehiculo', k+1, 'es', distancia_aux)
print('El camino optimo recorre una distancia = ', distancia_total)


# Se pone un salto de linea
print('')
# Se muestra el recorrido nodo a nodo
for k in K:
    # Se muestran los datos si se usa ese vehiculo
    if (u[k] != 0):
        for i in Ntot:
            for j in Ntot:
                if (x[i,j,k] != 0):
                    print('Vehiculo ', k+1, 'origen:', i+1, ' -> destino:', j+1, 
                          '. Distancia =', c[i,j])

# Se pone un salto de linea
print('')
# Se muestran los vehiculos que salen de cada deposito
for i in ND:
    num_salidas_aux = 0
    salidas_max_permitidas = 0
    for k in K:
        # Se muestran los datos si se usa ese vehiculo
        if (u[k] != 0):
            # Se buscan los movimientos desde el primer cliente (cuyo indice es (num_depositos - 1))
            # hasta el ultimo cliente (cuyo indice es (num_depositos + num_clientes))
            for j in Nv_x:
                if (x[i,j,k] != 0):
                    num_salidas_aux += 1
                    salidas_max_permitidas += DQ[i,k]
                    print('Num. deposito =', i+1, 
                          'Num. vehiculo =', k+1, 
                          '. Num salidas =', num_salidas_aux, 
                          '. Salidas maximas permitidas en ese deposito =',salidas_max_permitidas)

El recorrido optimo tiene gasto fijo en los vehiculos =  [200]
Vehiculo 2 . Coste fijo =  [100]
Vehiculo 3 . Coste fijo =  [100]

La distancia maxima permitida para un vehiculo es =  650
La distancia recorrida por el vehiculo 1 es 0
La distancia recorrida por el vehiculo 2 es 557
La distancia recorrida por el vehiculo 3 es 423
El camino optimo recorre una distancia =  980

Vehiculo  2 origen: 2  -> destino: 9 . Distancia = 191
Vehiculo  2 origen: 3  -> destino: 2 . Distancia = 10
Vehiculo  2 origen: 4  -> destino: 3 . Distancia = 220
Vehiculo  2 origen: 6  -> destino: 4 . Distancia = 50
Vehiculo  2 origen: 9  -> destino: 6 . Distancia = 86
Vehiculo  3 origen: 3  -> destino: 7 . Distancia = 100
Vehiculo  3 origen: 5  -> destino: 8 . Distancia = 195
Vehiculo  3 origen: 7  -> destino: 5 . Distancia = 62
Vehiculo  3 origen: 8  -> destino: 3 . Distancia = 66

Num. deposito = 2 Num. vehiculo = 2 . Num salidas = 1 . Salidas maximas permitidas en ese deposito = 1
Num. deposito = 3 Num. vehicul

## Productos. Reparto.

In [6]:
# Reparto de productos a cada cliente
print('Reparto de productos a cada cliente')
print('Num. cliente \t Num. vehiculo \t Num producto \t Cantidad producto')
for k in K:
    for j in Nv_y_d:
        for g in G:
            if (y[j,g,k] != 0 and d[j,g] != 0):
                print(j+1+num_depositos, '\t', k+1, '\t', g+1, '\t',d[j,g])

                    
# Se pone un salto de linea
print('')
# Numero de articulos transportados por un vehiculo
print('Numero de articulos transportados por un vehiculo')
print('Num. vehiculo\tNum. producto\tCantidad producto\tCantidad max producto permitida')
for k in K:
    # Se buscan los vehiculos que se usan
    if (u[k] != 0):
        for g in G:
            cantidad_aux = 0
            for j in Nv_y_d:
                if (y[j,g,k] != 0 and d[j,g] != 0):
                    cantidad_aux += d[j,g]
            print(k+1, '\t', g+1, '\t', cantidad_aux, '\t', Q[g])


# Se pone un salto de linea
print('')            
# Cantidad total de articulos demandados por todos los clientes
print('Cantidad total de articulos demandados por todos los clientes')
print('Num. producto\tCantidad total producto')
for g in G:
    cantidad_aux = 0
    for j in Nv_y_d:
        cantidad_aux += d[j,g]
    print(g+1, '\t', cantidad_aux)

    
# Se pone un salto de linea
print('')               
# Productos que salen de cada deposito
print('Productos que salen de cada deposito')
print("Depos.D \t Prod.G \t Cant.Prod.g \t Stock.Prod.g")
for i in ND:
    for g in G:
        cantidad_aux = 0
        for k in K:
            if (u[k] != 0 and DQ[i,k] != 0):
                for j in Nv_y_d:
                    if (y[j,g,k] != 0):
                        cantidad_aux += d[j,g]
                print(i+1, '\t', g+1, '\t', cantidad_aux, '\t', S[i,g])
                

Reparto de productos a cada cliente
Num. cliente 	 Num. vehiculo 	 Num producto 	 Cantidad producto
4 	 2 	 1 	 10
4 	 2 	 2 	 11
4 	 2 	 3 	 3
4 	 2 	 4 	 18
6 	 2 	 2 	 42
6 	 2 	 3 	 14
6 	 2 	 4 	 29
6 	 2 	 5 	 44
9 	 2 	 1 	 30
9 	 2 	 2 	 20
9 	 2 	 3 	 13
9 	 2 	 4 	 28
5 	 3 	 1 	 10
5 	 3 	 2 	 5
5 	 3 	 3 	 2
5 	 3 	 5 	 13
7 	 3 	 1 	 27
7 	 3 	 4 	 23
7 	 3 	 5 	 41
8 	 3 	 1 	 8
8 	 3 	 2 	 25
8 	 3 	 3 	 19
8 	 3 	 5 	 26

Numero de articulos transportados por un vehiculo
Num. vehiculo	Num. producto	Cantidad producto	Cantidad max producto permitida
2 	 1 	 40 	 [50]
2 	 2 	 73 	 [75]
2 	 3 	 30 	 [35]
2 	 4 	 75 	 [78]
2 	 5 	 44 	 [85]
3 	 1 	 45 	 [50]
3 	 2 	 30 	 [75]
3 	 3 	 21 	 [35]
3 	 4 	 23 	 [78]
3 	 5 	 80 	 [85]

Cantidad total de articulos demandados por todos los clientes
Num. producto	Cantidad total producto
1 	 85
2 	 103
3 	 51
4 	 98
5 	 124

Productos que salen de cada deposito
Depos.D 	 Prod.G 	 Cant.Prod.g 	 Stock.Prod.g
2 	 1 	 40 	 70
2 	 2 	 73 	